# Diffusion Limited Aggregation 

#### Author : B. Militzer, University of California, Berkeley 
#### Date   : Sept. 26, 2018

#### Read "The Science of Fractal Images", Ed. Peitgen and Saupe, p. 37 (1988)


In [15]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
#note, this function expects a matrix A[ix,iy] 
#and then displays so that A[:,0] is the lowest row of pixels
def display(A, B):
    maxX = A.shape[0]
    maxY = A.shape[1]
    
    m2 = np.max(B)
    B[A[:,:]==1] = m2
    
    m1 = np.min(B[B[:,:]>0])
    if(m1==m2): m1=m2/2
    B[B[:,:]==0] = m1*(m1/m2)**(1/10)
    
    D = np.zeros((maxY, maxX))
    for ix in range(0,maxX):
        for iy in range(0,maxY):
            D[maxY-1-iy,ix] = np.log(B[ix, iy])

    #Display the graphics outside of the notebook. 
    #On a PC, use '%matplotlib qt' instead.
    %matplotlib tk
    #NOTE: I am running Ubuntu and deleting this line worked for me. (having that line caused errors.)
    
    plt.rcParams['figure.figsize'] = [6, 6/maxX*maxY]
    plt.imshow(D); 
    plt.axis('off'); 
    plt.show()
    plt.draw()
    plt.pause(0.01)

In [17]:
nParticles = 100 * 1000
maxX = 200 +(5*7)
maxY = 40

In [18]:
# Initialize matrix containing all 2D grid points A(x,y)
# 0 <= x < maxX
# 0 <= y < maxY
# A(x,y)=0 ... site is empty
# A(x,y)>0 ... site is filled
A = np.zeros((maxX, maxY))

A[:,0] = 1
A[:,maxY-1] = 1
#print(A.transpose())

In [19]:
B = np.zeros((maxX, maxY))

#print(B.transpose())

In [20]:
p = 1
yStart = 20
xStart = maxX-1

In [21]:
def run_simulation(A, B):
    particles_finished = 0
    for i in range(0,nParticles):
        # Compute new starting point on the line y=yStart
        x  = xStart
        y  = yStart

        while True:
            xOrg = x
            yOrg = y

            r = np.random.random(); # Random float:  0.0 <= r < 1.0
            #based on the value of 'r', move the particle
            #left, right, up, or down and change x and y accordingly
            if r < 0.5: 
                x -= 1
            elif 0.5 <= r < 0.6: 
                x +=1
            elif 0.6 <= r < 0.8: 
                y += 1
            elif 0.8 <= r < 1:
                y -= 1

            #now apply periodic boundary conditions to 'x'
            if y == 0 or y == maxY-1:
                break
            if x < 0:
                particles_finished += 1
                #print(f'{i}th particle reached left side')
                break
            if x > maxX-1:
                continue


            if (A[x,y] == 1): 
                x = xOrg
                y = yOrg
                continue; # if this site has been taken try moving in a different direction

            #determine the x coordionates of the left and right neighbors
            #store them in 'xm' and 'xp' and apply periodic boundary conditions again
            xp = x+1
            xm = x-1
            yp = y+1
            ym = y-1

            if ym < 0:
                ym = maxY-1
            elif yp >= maxY:
                yp = 0
            elif xp >= maxX:
                xp = 0


            # Determine if any neighboring site is occupied
            # if that is the case, enter the following 'if' clause

            if (xm < 0 or A[xp,y] == 1 or A[xm,y] == 1 or A[x,yp] == 1 or A[x,ym] == 1): 
                B[x,y] += 1

                if(xm < 1):
                    particles_finished += 1
                    #print(f'{i}th particle reached left side')

                if (i%5000==0 and i != 0): 
                    p_curr = particles_finished/i
                    print(f'i= {i} \tCurrent probability={p_curr}')

                nNewParticlesPerFrame = 500 
                if (i%nNewParticlesPerFrame==0): 
                    display(A, B)

                break # particle was attached, break out of current loop and insert next one

        #if (x-1==0): 
        #    print(f'{i}th particle reached left side')
        #    particles_finished +=1
    return particles_finished

particles_finished = run_simulation(A, B)
print("Finished.")

p_finished = particles_finished / nParticles
print(f"Probability of reaching left side is {p_finished}")

display(A, B)
plt.savefig('final.png', bbox_inches='tight')  

i= 5000 	Current probability=0.5536
i= 10000 	Current probability=0.5492
i= 15000 	Current probability=0.5541333333333334
i= 20000 	Current probability=0.5565
i= 25000 	Current probability=0.5552
i= 30000 	Current probability=0.5555
i= 35000 	Current probability=0.5536857142857143
i= 40000 	Current probability=0.553
i= 45000 	Current probability=0.5526
i= 50000 	Current probability=0.55116
i= 55000 	Current probability=0.5521272727272727
i= 60000 	Current probability=0.5531833333333334
i= 65000 	Current probability=0.5538615384615385
i= 70000 	Current probability=0.5536714285714286
i= 75000 	Current probability=0.5540266666666667
i= 80000 	Current probability=0.5543875
i= 85000 	Current probability=0.5535176470588236
i= 90000 	Current probability=0.5531333333333334
i= 95000 	Current probability=0.5535894736842105
Finished.
Probability of reaching left side is 0.55315


## Probabilities:
**5000 Particles**:  0.5482

**10000 Particles**: 0.5573

**15000 Particles**: 0.5569333333333333

**20000 Particles**: 0.05514

**25000 Particles**: 0.54916, 0.54992,

**50000 Particles**: 0.5547, 0.54852

**100000 Particles** 0.55144, 0.55137, 0.5514

## Final Success Probability:
### 55.14%

# Add Obstacle

In [22]:
# Initialize matrix containing all 2D grid points A(x,y)
# 0 <= x < maxX
# 0 <= y < maxY
# A(x,y)=0 ... site is empty
# A(x,y)>0 ... site is filled
A = np.zeros((maxX, maxY))

A[:,0] = 1
A[:,maxY-1] = 1

# Add obstacle:
# Chosen obstacle 7. SID: 3032102477
center_x = maxX // 2
center_y = maxY // 2
obst_width = 25
obst_height = 8
obst_thickness = 2
obst_min_x = center_x - obst_width
obst_max_x = center_x + obst_width
obst_min_y = center_y - obst_height - obst_thickness
obst_max_y = center_y + obst_height + obst_thickness

for i in range(obst_min_x, obst_max_x):
    for j in range(obst_min_y, obst_max_y):
        if j < obst_min_y + obst_thickness or j > obst_max_y - obst_thickness:
            A[i, j] = 1
#print(A.transpose())

B = np.zeros((maxX, maxY))

#print(B.transpose())

p = 1
yStart = 20
xStart = maxX-1


In [23]:

particles_finished = run_simulation(A, B)
print("Finished.")

p_finished = particles_finished / nParticles
print(f"Probability of reaching left side is {p_finished}")

display(A, B)

i= 5000 	Current probability=0.1718
i= 10000 	Current probability=0.1648
i= 15000 	Current probability=0.16433333333333333
i= 20000 	Current probability=0.1664
i= 25000 	Current probability=0.1664
i= 30000 	Current probability=0.1683
i= 35000 	Current probability=0.16831428571428572
i= 40000 	Current probability=0.168675
i= 45000 	Current probability=0.16773333333333335
i= 50000 	Current probability=0.16762
i= 55000 	Current probability=0.16732727272727274
i= 60000 	Current probability=0.16728333333333334
i= 65000 	Current probability=0.16693846153846154
i= 70000 	Current probability=0.16615714285714286
i= 75000 	Current probability=0.16606666666666667
i= 80000 	Current probability=0.1659875
i= 85000 	Current probability=0.16581176470588235
i= 90000 	Current probability=0.1660888888888889
i= 95000 	Current probability=0.16590526315789475
Finished.
Probability of reaching left side is 0.16604


### Probability With obstacle:
#### 16.8% +/- 0.2%

# Obstacle with double height:

In [24]:
# Initialize matrix containing all 2D grid points A(x,y)
# 0 <= x < maxX
# 0 <= y < maxY
# A(x,y)=0 ... site is empty
# A(x,y)>0 ... site is filled
A = np.zeros((maxX, maxY))

A[:,0] = 1
A[:,maxY-1] = 1

# Add obstacle:
# Chosen obstacle 7. SID: 3032102477
center_x = maxX // 2
center_y = maxY // 2
obst_width = 25
obst_height = 8
obst_thickness = 4
obst_min_x = center_x - obst_width
obst_max_x = center_x + obst_width
obst_min_y = center_y - obst_height - obst_thickness
obst_max_y = center_y + obst_height + obst_thickness

for i in range(obst_min_x, obst_max_x):
    for j in range(obst_min_y, obst_max_y):
        if j < obst_min_y + obst_thickness or j > obst_max_y - obst_thickness:
            A[i, j] = 1
#print(A.transpose())

B = np.zeros((maxX, maxY))

#print(B.transpose())

p = 1
yStart = 20
xStart = maxX-1


particles_finished = run_simulation(A, B)

print("Finished.")

p_finished = particles_finished / nParticles
print(f"Probability of reaching left side is {p_finished}")

display(A, B)

i= 5000 	Current probability=0.1724
i= 10000 	Current probability=0.1728
i= 15000 	Current probability=0.17006666666666667
i= 20000 	Current probability=0.16745
i= 25000 	Current probability=0.16788
i= 30000 	Current probability=0.16863333333333333
i= 35000 	Current probability=0.16894285714285714
i= 40000 	Current probability=0.168825
i= 45000 	Current probability=0.1680888888888889
i= 50000 	Current probability=0.1669
i= 55000 	Current probability=0.16736363636363635
i= 60000 	Current probability=0.16738333333333333
i= 65000 	Current probability=0.16709230769230768
i= 70000 	Current probability=0.16712857142857143
i= 75000 	Current probability=0.16716
i= 80000 	Current probability=0.1669
i= 85000 	Current probability=0.16618823529411764
i= 90000 	Current probability=0.16642222222222222
i= 95000 	Current probability=0.1664
Finished.
Probability of reaching left side is 0.16662


### Probability with double height:
#### 16.6% +/- 0.2%
The Probability is now slightly less, since there is more surface area for the particles to get stuck on, therefore less particles get through to the end of the pipe.

# Obstacle with double width

In [25]:
# Initialize matrix containing all 2D grid points A(x,y)
# 0 <= x < maxX
# 0 <= y < maxY
# A(x,y)=0 ... site is empty
# A(x,y)>0 ... site is filled
A = np.zeros((maxX, maxY))

A[:,0] = 1
A[:,maxY-1] = 1

# Add obstacle:
# Chosen obstacle 7. SID: 3032102477
center_x = maxX // 2
center_y = maxY // 2
obst_width = 50
obst_height = 8
obst_thickness = 2
obst_min_x = center_x - obst_width
obst_max_x = center_x + obst_width
obst_min_y = center_y - obst_height - obst_thickness
obst_max_y = center_y + obst_height + obst_thickness

for i in range(obst_min_x, obst_max_x):
    for j in range(obst_min_y, obst_max_y):
        if j < obst_min_y + obst_thickness or j > obst_max_y - obst_thickness:
            A[i, j] = 1
#print(A.transpose())

B = np.zeros((maxX, maxY))

#print(B.transpose())

p = 1
yStart = 20
xStart = maxX-1


particles_finished = run_simulation(A, B)

print("Finished.")

p_finished = particles_finished / nParticles
print(f"Probability of reaching left side is {p_finished}")

display(A, B)

i= 5000 	Current probability=0.0808
i= 10000 	Current probability=0.0806
i= 15000 	Current probability=0.0816
i= 20000 	Current probability=0.08185
i= 25000 	Current probability=0.08176
i= 30000 	Current probability=0.0807
i= 35000 	Current probability=0.08062857142857142
i= 40000 	Current probability=0.080175
i= 45000 	Current probability=0.07937777777777778
i= 50000 	Current probability=0.07964
i= 55000 	Current probability=0.07974545454545455
i= 60000 	Current probability=0.07946666666666667
i= 65000 	Current probability=0.07966153846153846
i= 70000 	Current probability=0.07938571428571428
i= 75000 	Current probability=0.0792
i= 80000 	Current probability=0.079
i= 85000 	Current probability=0.07905882352941176
i= 90000 	Current probability=0.07898888888888889
i= 95000 	Current probability=0.07886315789473684
Finished.
Probability of reaching left side is 0.07926


### Probability with double width:
#### 7.9% +/- 0.2%
The Probability is now significantly less because the obstacle now starts closer to the start of the path of particles. This means a large number of them enters between the obstacles, and the obstacles form 3 smaller pseudo-pipes. Since the individual pipes a smaller, its harder for the particles to get through the pipes (since it doesnt take as much "work" for the particle to bump and get stuck on one of the walls.)